In [10]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from sklearn.metrics import accuracy_score
import os
import random
import datetime
import matplotlib.pyplot as plt
import tensorflow_hub as hub

In [19]:
classes = pd.read_csv("../artifacts/data/labels.csv")
classes

,ClassId,Name
0,0,Speed limit (5km/h)
1,1,Speed limit (15km/h)
2,2,Speed limit (30km/h)
3,3,Speed limit (40km/h)
4,4,Speed limit (50km/h)
5,5,Speed limit (60km/h)
6,6,Speed limit (70km/h)
7,7,speed limit (80km/h)
8,8,Dont Go straight or left
9,9,Dont Go straight or Right


In [20]:
# How many images in each folder
import os

# Walk through 10 percent data directory and list # of files
for dirpath, dirnames, filenames in os.walk("../artifacts/data/images"):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in {dirpath}")

There are 2 directories and 0 images in ../artifacts/data/images
There are 58 directories and 0 images in ../artifacts/data/images\TEST
There are 0 directories and 14 images in ../artifacts/data/images\TEST\0
There are 0 directories and 12 images in ../artifacts/data/images\TEST\1
There are 0 directories and 60 images in ../artifacts/data/images\TEST\10
There are 0 directories and 130 images in ../artifacts/data/images\TEST\11
There are 0 directories and 22 images in ../artifacts/data/images\TEST\12
There are 0 directories and 92 images in ../artifacts/data/images\TEST\13
There are 0 directories and 12 images in ../artifacts/data/images\TEST\14
There are 0 directories and 36 images in ../artifacts/data/images\TEST\15
There are 0 directories and 76 images in ../artifacts/data/images\TEST\16
There are 0 directories and 84 images in ../artifacts/data/images\TEST\17
There are 0 directories and 0 images in ../artifacts/data/images\TEST\18
There are 0 directories and 0 images in ../artifacts

In [21]:
# Setup data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SHAPE = (64, 64)
BATCH_SIZE = 32

train_dir = "../artifacts/data/images/TRAIN/"
test_dir = "../artifacts/data/images/TEST/"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training images:")
train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=IMG_SHAPE,
                                               batch_size=BATCH_SIZE,
                                               class_mode="categorical")
print("Testing images:")
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=IMG_SHAPE,
                                             batch_size=BATCH_SIZE,
                                             class_mode="categorical")

Training images:
Found 4170 images belonging to 58 classes.
Testing images:
Found 1994 images belonging to 58 classes.


In [22]:
# Create a TensorBoard callback (functionized because we need to create a new one for each model)
import datetime
import tensorflow as tf

def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    print(f"Saving Tensorboard log files to: {log_dir}")
    return tensorboard_callback

In [23]:
# Let's compare the following two models
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5"
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [24]:
# Let's create a create_model() function to create a model from a URL
def create_model(model_url, num_classes=58):
    """
    Takes a tensorflow hub URL and create a keras sequential model with it.
    
    Args:
        model_url (str): A TensorFlow Hub feature extraction URL
        num_classes (int): Number of output neurons in the output layer, should be equal to number of target classes,
        default 10.
    
    Returns:
        An uncompiled Keras Sequential model with model_url as feature extractor layer and Dense output layer with 
        num_classes output neurons.

    """
    # Download the pretrained model and save it as Keras layer
    feature_extraction_layer = hub.KerasLayer(model_url,
                                              trainable=False, # Freeze already learnt patterns
                                              name="feature_extraction_layer",
                                              input_shape=IMG_SHAPE+(3,)) # Adds another dimension (i.e channel) as it needs to be (rows, cols, channels)
    
    # Create our model
    model = tf.keras.Sequential([
        feature_extraction_layer,
        layers.Dense(num_classes, activation="softmax", name="output_layer")
    ])
    
    return model

In [29]:
# Create Resnet model
resnet_model = create_model(resnet_url,
                            num_classes=train_data.num_classes)

In [ ]:
# Compile our resnet model
resnet_model.compile(loss="categorical_crossentropy",
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])

In [ ]:
resnet_model.summary()

In [ ]:
# Fit the resnet model to the data
resnet_history = resnet_model.fit(train_data,
                                  epochs=5,
                                  steps_per_epoch=len(train_data),
                                  validation_data=test_data,
                                  validation_steps=int(0.15*len(test_data)),
                                  callbacks=[create_tensorboard_callback(dir_name="D:\\01_AIML\\02_Notebooks\\02_Projects\\05_Traffic_sign_classification\\callbacks",
                                                                         experiment_name="resnet50V2"
                                                                        )])

In [ ]:
# Let's evaluate it over all of the test data
resnet_results = resnet_model.evaluate(test_data)
resnet_results

In [ ]:
# Let's create a function to plot our loss curves
# Note: You could put this function into a script called "helper.py" and import it when you need it....
import matplotlib.pyplot as plt

def plot_loss_curves(history):
    """
    Returns separate loss curves for training & validation curves
    Args:
        history: TensorFlow History object
    Returns:
        Plots of training/validation loss and accuracy metrics
    """
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    accuracy = history.history["accuracy"]
    val_accuracy = history.history["val_accuracy"]
    
    epochs = range(len(history.history["loss"]))
    
    #Plot loss
    plt.plot(epochs, loss, label="training_loss")
    plt.plot(epochs, val_loss, label="val_loss")
    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    
    #Plot Accuracy
    plt.figure()
    plt.plot(epochs, accuracy, label="training_accuracy")
    plt.plot(epochs, val_accuracy, label="val_accuracy")
    plt.title("Accuracy")
    plt.xlabel("Epochs")
    plt.legend() 

In [ ]:
plot_loss_curves(resnet_history)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
import numpy as np

In [ ]:
# Create a checkpoint callback
checkpoint_path = "callbacks/traffic_resnet"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True,
                                                         monitor='val_accuracy',
                                                         save_best_only=True)